## Spam message classifier 

In this noebook we're using kaggle's dataset to classify weather a sms is spam or not
It's very easy to understand. So let's get started

In [ ]:
import pandas as pd

In [ ]:
# Let's first import the data
messages = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv", encoding = "ISO-8859-1")

In [ ]:
messages

In [ ]:
messages.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [ ]:
messages

We'll use nltk library and perform some data cleaning

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['v2'][i])
  review = review.lower()
  review = review.split()

  review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


## Use tf idf to convert words into vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features = 2500)
x = tf.fit_transform(corpus).toarray()

In [ ]:
y = pd.DataFrame(messages['v1'])
y

In [ ]:
## now for converting all string values into categorial values
for labels, contents in y.items():
    if pd.api.types.is_string_dtype(contents):
        y['v1'] = contents.astype('category').cat.as_ordered()

In [ ]:
y.v1.cat.codes

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x , y, test_size = 0.2, random_state = 2)

## Useing Naive_bayes to classify

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(xtrain, ytrain)
ypreds = spam_detect_model.predict(xtest)

In [ ]:
ypreds

In [ ]:
ytest

In [ ]:
from sklearn.metrics import classification_report
classification_report(ytest, ypreds)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, ypreds)

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score
ac_s = accuracy_score(ytest, ypreds)
ac_s

Yeeh!! got 97% accuracy by simple modelling